In [4]:
import wbgapi as wb
import pandas as pd

class DataPipeline():

    def __init__(self, indicators, year):
        self.indicators = indicators
        self.year = year

        self.happiness_data = pd.read_csv('../data/happiness/happiness.csv').drop('Country', axis=1)
        self.valid_countries = self.happiness_data['ISO_A3'].unique()

        wb.db = 2
        # print(wb.series.info())

    def pull_wb_data(self):
        data = wb.data.DataFrame(self.indicators, time=self.year)
        return data
    
    def get_data(self):
        features = self.pull_wb_data()
        features = features.reset_index()
        features = features.rename(columns={features.columns[0]: 'ISO_A3'})
        features = features[features['ISO_A3'].isin(self.valid_countries)]
        merged = pd.merge(features, self.happiness_data, on='ISO_A3')
        return merged
dp = DataPipeline(['NY.GDP.PCAP.CD', 'SP.POP.TOTL'], 2022)
data = dp.get_data()
print(data.head())

print(data.isna().sum())
data = data.dropna()


    


  ISO_A3  NY.GDP.PCAP.CD  SP.POP.TOTL  Happiness score
0    AFG      352.603733   41128771.0           2.4038
1    ALB     6810.114041    2777689.0           5.1988
2    ARE    53707.980081    9441129.0           6.5760
3    ARG    13650.604629   46234830.0           5.9670
4    ARM     7018.051504    2780469.0           5.3986
ISO_A3             0
NY.GDP.PCAP.CD     2
SP.POP.TOTL        0
Happiness score    0
dtype: int64


In [5]:
from model import Model

X = data.iloc[:, 1:-1].to_numpy()
y = data.iloc[:, -1].to_numpy()

model = Model(model_type='rdf', model_choice='reg', tuning_strategy='random')
model.make_quick_model()
model.make_full_model(X, y)

/home/jcdutoit/miniconda3/envs/acme/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jcdutoit/Documents/ACME/Vol3/Vol3FallProj/Joseph_workspace/model.py:213: UserWarning: ACHTUNG! Your specified model will be made but cannot be hypertuned.
  warnings.warn(message="ACHTUNG! Your specified model will be made but cannot be hypertuned.")


Now making RandomForestRegressor...
Training and hypertuning using Exhaustive Search...


/home/jcdutoit/miniconda3/envs/acme/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Training completed.
The best oob_score is 0.6267120238989977
Best model and hyperparameters added as attribute to class.
